In [1]:
import argparse
import numpy as np
from softlearning.environments.adapters.gym_adapter import GymAdapter
import gym
import dsuite
import imageio
import pickle
import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import skimage

%matplotlib inline

/home/abhigupta/anaconda/envs/vice/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
W0123 17:32:34.096828 140195668342528 __init__.py:690] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


In [2]:
exp_name = 'free_screw_lighting_fix'

In [3]:
cur_dir = os.getcwd()
directory = os.path.join(cur_dir, exp_name)
if not os.path.exists(directory):
    os.makedirs(directory)
directory

'/home/abhigupta/Libraries/vice/notebooks/free_screw_lighting_fix'

In [4]:
images = True

image_shape = (32, 32, 3)
NUM_TOTAL_EXAMPLES, ROLLOUT_LENGTH, STEPS_PER_SAMPLE = 300, 15, 3
env_kwargs = {
    'pixel_wrapper_kwargs': {
        'pixels_only': False,
        'normalize': False,
        'render_kwargs': {
            'width': image_shape[0],
            'height': image_shape[1],
            'camera_id': -1,
            'box_warp': True,
        },
    },
    'observation_keys': (
        'pixels',
        'claw_qpos',
        'last_action',
    ),
    'device_path': '/dev/ttyUSB0',
    'camera_config': {
        'topic': '/kinect2_001161563647/qhd/image_color',
        'image_shape': (256, 256, 3),
    }
}
env = GymAdapter(
    domain='DClaw',
    task='TurnFreeValve3Hardware-v0',
    **env_kwargs
) 

CREATING NEW IMAGE_SERVICE
subscribing to:  /kinect2_001161563647/qhd/image_color
No image found yet.
No image found yet.


/home/abhigupta/anaconda/envs/vice/lib/python3.7/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/home/abhigupta/anaconda/envs/vice/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
obs = env.reset()

In [6]:
obs['claw_qpos'].astype(int)

array([ 0, -1,  1,  1, -1,  1,  0, -1,  1])

In [10]:
from dsuite.dclaw.base_env import DEFAULT_CLAW_RESET_POSE
DEFAULT_CLAW_RESET_POSE.astype(int)

array([ 0, -1,  1,  0, -1,  1,  0, -1,  1])

In [ ]:
observation, _, _, _ = env.step(env.action_space.sample())
img_obs = observation['pixels']
print(img_obs.shape)
plt.imshow(img_obs)
plt.show()

In [ ]:
img_obs

In [ ]:
from skimage import transform
rect = np.array([[5, 10], [27, 10], [0, 26], [31, 26]], np.float32)
dst = np.array([[0, 0], [31, 0], [0, 31], [31, 31]], np.float32)
tform3 = transform.ProjectiveTransform()
tform3.estimate(dst, rect)

In [ ]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [ ]:
import cv2

M = cv2.getPerspectiveTransform(rect, dst)
warped = cv2.warpPerspective(img_obs, M, (32, 32))
plt.imshow(warped)

In [ ]:
warped = transform.warp(img_obs, tform3, output_shape=(32, 32))
plt.imshow(warped)

In [6]:
observations = []
num_positives = 0

In [7]:
# reset the environment
while num_positives <= NUM_TOTAL_EXAMPLES:
    observation = env.reset()
    print("Resetting environment...")
    t = 0
    while t < ROLLOUT_LENGTH:
        action = env.action_space.sample()
        
        for _ in range(STEPS_PER_SAMPLE):
            observation, _, _, _ = env.step(action)

#         input('C TO CONTINUE: ')
        observation, _, _, _ = env.step(action)
        
        img_obs = observation['pixels']
        plt.imshow(img_obs)
        plt.show()
        
        print(f'# {num_positives}')
        save = input('S TO SAVE, D TO DELETE: ')
        if save == 'S' or save == 's':        
            observations.append(observation)
            num_positives += 1
            print(num_positives)
            t += 1
        clear_output()
    
goal_examples = {
    key: np.concatenate([
        obs[key][None] for obs in observations
    ], axis=0)
    for key in observations[0].keys()
}
with open(os.path.join(directory, 'positives.pkl'), 'wb') as file:
    pickle.dump(goal_examples, file)

In [ ]:
with open('/home/abhigupta/Libraries/vice/goal_pools/free_screw_goal_images_black_box_more_friction/positives.pkl', 'rb') as f:
    data = pickle.load(f)